In [2]:
import pandas as pd
df_raw_vars = pd.read_parquet("raw_variables.parquet.gzip")
df_raw_meds = pd.read_parquet("raw_meds.parquet.gzip")
df_binned_vars = pd.read_parquet("binned_variables.parquet.gzip")
df_binned_meds = pd.read_parquet("binned_meds.parquet.gzip")

In [4]:
df_raw_meds[df_raw_meds["med"].str.contains("insulin", case=False, na=False)]

,pat_id,csn,dob,med_id,med,mar_action,mar_time,dose,dose_unit
9691,c3d8b294e89051e8cf7cf70e0d7323a18603d0e484d87e...,221790d0bf39ed049f3db20cb8ecf438ae295d2113b821...,2005-08-07,600644,INSULIN (NOVOLIN R) STANDARD DRIP IN 100 ML NS,Dose Change,2017-10-29 03:14:00,0.0950,Units/kg/hr
10762,78bef7922744b8ea02f851a13c236518c971e7912c3bd7...,5b279df747d7e7c0c244a841f10ef9dc1b688f2554a64a...,2011-02-22,600644,INSULIN (NOVOLIN R) STANDARD DRIP IN 100 ML NS,Dose Change,2017-02-01 21:37:00,0.3000,Units/kg/hr
11416,26e7e11cf077491004ccd0c7591c9f4b9cf0e4c3532ce0...,03261795f188fdcc1d8a0c90fc106c9cf61c87f18b7bce...,2006-10-26,50552,INSULIN LISPRO (U-100) 100 UNIT/ML SUBCUTANEOU...,Given,2019-01-13 15:33:00,10.0000,Units
14048,7a93b0dac04b0432ab794951f9c1a6bf3ac871163f2251...,257cab76b4da06adda9a308418caf030d4a5199f0c12e2...,2003-04-28,50552,INSULIN LISPRO (U-100) 100 UNIT/ML SUBCUTANEOU...,Given,2021-01-30 09:05:00,2.0000,Units
23615,514c92c78ea186e774020cac3415a538f1ef2a59a7821c...,aabaebdee6e9d41bacb3c51805abe235275a4b169c1cbd...,2009-06-02,110591,INSULIN GLARGINE (U-100) 100 UNIT/ML (3 ML) SU...,Given,2021-07-20 13:00:00,15.0000,Units
...,...,...,...,...,...,...,...,...,...
7097315,7a93b0dac04b0432ab794951f9c1a6bf3ac871163f2251...,257cab76b4da06adda9a308418caf030d4a5199f0c12e2...,2003-04-28,110591,INSULIN GLARGINE (U-100) 100 UNIT/ML (3 ML) SU...,Given,2021-02-26 21:31:00,32.0000,Units
7102478,caf08c61365906c26c9f0c9e560d1a44aaa48f2f66b92c...,1889491a8b00d4a804837f397a7a2350eff638186e8ed8...,2001-11-21,600644,INSULIN (NOVOLIN R) STANDARD DRIP IN 100 ML NS,Rate Change,2018-08-06 08:20:00,0.0500,Units/kg/hr
7108284,38230aaa8bb5a53ca4543b47a4d65a18ff1ef5d5e9714f...,8ddb7a445ba92e7f84f8f4a1fe25132e4fe329a2dece64...,2003-06-07,50552,INSULIN LISPRO (U-100) 100 UNIT/ML SUBCUTANEOU...,Given,2021-04-22 08:06:00,8.0000,Units
7113422,3feaee7a2a8bc0a82279ae647cbc36f1e9f252628c48e2...,429072dfc3a78ce9cf988676f034ce9c01376ecb5bfa0c...,2003-12-20,28166,INSULIN LISPRO (U-100) 100 UNIT/ML SUBCUTANEOU...,Given,2016-11-21 13:09:00,15.0000,Units


In [22]:
df_raw_vars.info() 
df_raw_meds.info()
df_binned_vars.info()
df_binned_meds.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1995804 entries, 0 to 1995803
Data columns (total 7 columns):
 #   Column         Dtype         
---  ------         -----         
 0   pat_id         object        
 1   csn            object        
 2   dob            datetime64[ns]
 3   variable_id    int64         
 4   variable_name  object        
 5   recorded_time  datetime64[ns]
 6   value          object        
dtypes: datetime64[ns](2), int64(1), object(4)
memory usage: 106.6+ MB
<class 'pandas.core.frame.DataFrame'>
Index: 91798 entries, 149 to 7120860
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   pat_id      91798 non-null  object        
 1   csn         91798 non-null  object        
 2   dob         91798 non-null  datetime64[ns]
 3   med_id      91798 non-null  int64         
 4   med         91798 non-null  object        
 5   mar_action  91798 non-null  object        
 6   mar_ti

In [2]:
import pandas as pd, numpy as np
from pathlib import Path          # ← NEW

ID_COLS   = ["pat_id", "csn", "dob"]
VITAL_KW  = ["hr","spo2","bp","map","resp","temp","fio2",
             "o2sat","sbp","dbp","pulse"]
LAB_KW    = ["bun","creat","lact","bilirubin","alt","ast","platelet","wbc",
             "hgb","hct","glucose","ph","pco2","po2","na","k","cl","co2"]

def add_hour_bin(df, abs_col):
    df = df.copy()
    df["event_time"] = pd.to_datetime(df[abs_col], errors="coerce")
    df["hour_bin"]   = df["event_time"].dt.floor("H")
    return df

vars_hour = add_hour_bin(df_raw_vars, "recorded_time")
meds_hour = add_hour_bin(df_raw_meds, "mar_time")

def collapse_hour(df):
    keep = {c:"last" for c in df.columns
            if c not in ID_COLS+["event_time","hour_bin"]}
    return df.groupby(ID_COLS+["hour_bin"], as_index=False).agg(keep)

vars_hour = collapse_hour(vars_hour)
meds_hour = collapse_hour(meds_hour)

merged = vars_hour.merge(
            meds_hour, on=ID_COLS+["hour_bin"], how="outer",
            suffixes=("","_med")
         )

# Example event flag: “insulin was given this hour”
merged["given_insulin"] = (
    merged["med"]
      .str.contains("insulin", case=False, na=False)
      .astype("int8")
)

# Zero‑fill ALL *_event, given_* or mar_action* flags
event_cols = [c for c in merged.columns
              if c.startswith("given_") or c.endswith("_event") or
                 "mar_action" in c]
merged[event_cols] = merged[event_cols].fillna(0)

def ffill_bounded(group, cols, hours):
    for col in cols:
        last_val, last_t = np.nan, None
        buf = []
        for t, v in zip(group["hour_bin"], group[col]):
            if pd.notna(v):
                last_val, last_t = v, t
            elif last_t is not None and (t - last_t) <= pd.Timedelta(hours=hours):
                v = last_val
            buf.append(v)
        group[col] = buf
    return group

num_cols = merged.select_dtypes("number").columns.difference(ID_COLS+["hour_bin"])
vitals   = [c for c in num_cols if any(k in c.lower() for k in VITAL_KW)]
labs     = [c for c in num_cols if any(k in c.lower() for k in LAB_KW)]

merged = (merged.sort_values(ID_COLS+["hour_bin"])
                .groupby(["pat_id","csn"], group_keys=False)
                .apply(lambda g: ffill_bounded(g, vitals, 24)))
merged = (merged.groupby(["pat_id","csn"], group_keys=False)
                .apply(lambda g: ffill_bounded(g, labs, 32)))

for col in merged.columns.difference(ID_COLS+["hour_bin"]):
    if pd.api.types.is_numeric_dtype(merged[col]):
        merged[col].fillna(merged[col].median(), inplace=True)
    else:
        merged[col].fillna("missing", inplace=True)

cats  = merged.select_dtypes("object").columns.difference(["hour_bin"])
final = pd.get_dummies(merged, columns=cats, drop_first=False)

# ─── 9  drop hour_bin **before** NaN check ─────────────────────────────
final.drop(columns=["hour_bin"], inplace=True)

# ─── 10 final sanity  ──────────────────────────────────────────────────
nulls = final.isna().sum()
if (nulls > 0).any():
    print("🚨 Remaining NaNs by column:\n", nulls[nulls > 0].head())
    raise ValueError("NaNs remain after all imputation!")

print("✅  shape:", final.shape)

/tmp/ipykernel_3535094/2900459914.py:72: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'missing' has dtype incompatible with datetime64[ns], please explicitly cast to a compatible dtype first.
  merged[col].fillna("missing", inplace=True)


✅  shape: (230225, 227988)


In [ ]:
final.to_csv("processed_data.csv.gz", compression="gzip", index=False)

In [ ]:
# ================================================================
RAW_VAR_PATH  = "raw_variables.parquet.gzip"        # vitals + labs
RAW_MED_PATH  = "raw_meds.parquet.gzip"             # medications
TOP_K_MEDS    = 200     # multi‑hot encode K most common drugs
VITAL_KW      = ["hr","spo2","bp","map","resp","temp","fio2",
                 "o2sat","sbp","dbp","pulse"]
LAB_KW        = ["bun","creat","lact","bilirubin","alt","ast",
                 "platelet","wbc","hgb","hct","glucose","ph",
                 "pco2","po2","na","k","cl","co2"]
HOURS_FFILL_VITALS = 24  # LOCF horizon for vitals
HOURS_FFILL_LABS   = 32  # LOCF horizon for labs
T_MAX         = 48       # length of each sequence (hours)
OUT_PREFIX    = "48hrs"  # appears in output filenames
# ---------------------------------------------------------------

import pandas as pd, numpy as np, pickle, gzip, os
from tqdm.auto import tqdm
from pathlib import Path

ID_COLS = ["pat_id", "csn", "dob"]

# ================================================================
import pandas as pd
df_raw_vars = pd.read_parquet("raw_variables.parquet.gzip")
df_raw_meds = pd.read_parquet("raw_meds.parquet.gzip")
# ================================================================
df_vars = pd.read_parquet("raw_variables.parquet.gzip")
df_meds =  pd.read_parquet("raw_meds.parquet.gzip")

# Cast numeric strings to numbers
df_vars["value"] = pd.to_numeric(df_vars["value"], errors="coerce")
df_meds["dose"]  = pd.to_numeric(df_meds["dose"],  errors="coerce")

# ================================================================
# 2 ── BUILD HOURLY GRID FOR VITALS / LABS -----------------------
# ================================================================
df_vars["hour_bin"] = pd.to_datetime(df_vars["recorded_time"],
                                     errors="coerce").dt.floor("H")

# keep only rows with keywords of interest
sel = df_vars["variable_name"].str.contains("|".join(VITAL_KW+LAB_KW),
                                            case=False, na=False)
df_vars = df_vars[sel].copy()

pivot_vars = (df_vars
              .pivot_table(index=ID_COLS + ["hour_bin"],
                           columns="variable_name",
                           values="value",
                           aggfunc="last")      # last measurement in the hour
              .reset_index())

vital_cols = [c for c in pivot_vars.columns
              if any(k in c.lower() for k in VITAL_KW)]
lab_cols   = [c for c in pivot_vars.columns
              if c not in ID_COLS+["hour_bin"]+vital_cols]

# ================================================================
# 3 ── MULTI‑HOT ENCODE TOP‑K MEDS (+ given_insulin) -------------
# ================================================================
df_meds["hour_bin"] = pd.to_datetime(df_meds["mar_time"],
                                     errors="coerce").dt.floor("H")

top_meds = (df_meds["med"]
            .value_counts()
            .head(TOP_K_MEDS)
            .index)

df_meds_flag = (df_meds[df_meds["med"].isin(top_meds)]
                .assign(flag=1)
                .pivot_table(index=ID_COLS + ["hour_bin"],
                             columns="med",
                             values="flag",
                             aggfunc="max",
                             fill_value=0)
                .reset_index())

med_cols = [c for c in df_meds_flag.columns
            if c not in ID_COLS+["hour_bin"]]

# ================================================================
# 4 ── MERGE & CREATE given_insulin BEFORE IMPUTATION ------------
# ================================================================
hourly = (pivot_vars
          .merge(df_meds_flag, on=ID_COLS+["hour_bin"], how="outer"))

hourly["given_insulin"] = (
    hourly.get("Insulin", pd.Series(0, index=hourly.index))
          .fillna(0)
          .astype("int8")
)

if "given_insulin" not in med_cols:
    med_cols += ["given_insulin"]


print(f"📊 Merged hourly shape: {hourly.shape} — rows: {len(hourly):,}, columns: {hourly.shape[1]}")

# ================================================================
# 5 ── LOCF WITH BOUNDED WINDOWS ---------------------------------
# ================================================================
def ffill_bounded(group, cols, horizon):
    for col in cols:
        last_val, last_t, buf = np.nan, None, []
        for t, v in zip(group["hour_bin"], group[col]):
            if pd.notna(v):
                last_val, last_t = v, t
            elif last_t is not None and (t - last_t) <= pd.Timedelta(hours=horizon):
                v = last_val
            buf.append(v)
        group[col] = buf
    return group

hourly = (hourly.sort_values(ID_COLS+["hour_bin"])
                .groupby(["pat_id","csn"], group_keys=False)
                .apply(lambda g: ffill_bounded(g, vital_cols, HOURS_FFILL_VITALS))
                .groupby(["pat_id","csn"], group_keys=False)
                .apply(lambda g: ffill_bounded(g, lab_cols, HOURS_FFILL_LABS)))

# ================================================================
# 6 ── HANDLE REMAINING NANS -------------------------------------
# ================================================================
hourly[vital_cols+lab_cols] = hourly[vital_cols+lab_cols].apply(
    lambda col: col.fillna(col.median())
)
hourly[med_cols] = hourly[med_cols].fillna(0)

# Any residual object columns → "missing", then one‑hot
cat_cols = hourly.select_dtypes("object").columns.difference(["hour_bin"])
group_keys = hourly[["pat_id", "csn", "hour_bin"]]  # optionally keep 'dob' too
hourly[cat_cols] = hourly[cat_cols].fillna("missing")
hourly = pd.get_dummies(hourly, columns=cat_cols, drop_first=False)
hourly[["pat_id", "csn", "hour_bin"]] = group_keys

# ================================================================
# 7 ── LIMIT TO FIRST T_MAX HOURS PER ENCOUNTER ------------------
# ================================================================
def trim_first_hours(g):
    g = g.sort_values("hour_bin")
    return g.iloc[:T_MAX]

hourly = (hourly.groupby(["pat_id","csn"], group_keys=False)
                .apply(trim_first_hours))

# ================================================================
# 8 ── SPLIT INTO CONTINUOUS & DISCRETE --------------------------
# ================================================================
continuous_cols = vital_cols + lab_cols          # float32
discrete_cols   = med_cols                       # int8  (0/1)

hourly[continuous_cols] = hourly[continuous_cols].astype("float32")
hourly[discrete_cols]   = hourly[discrete_cols].astype("int8")

vital_tensors, med_tensors, lengths = [], [], []

for _, grp in tqdm(hourly.groupby(["pat_id","csn"]), desc="packing"):
    x_c = grp[continuous_cols].to_numpy()        # (L_i, d_c)
    x_d = grp[discrete_cols].to_numpy()          # (L_i, d_d)
    lengths.append(len(x_c))

    if len(x_c) < T_MAX:                         # zero‑pad shorter stays
        pad_c = np.zeros((T_MAX-len(x_c), len(continuous_cols)), dtype=np.float32)
        pad_d = np.zeros((T_MAX-len(x_d), len(discrete_cols)),   dtype=np.int8)
        x_c   = np.vstack([x_c, pad_c])
        x_d   = np.vstack([x_d, pad_d])

    vital_tensors.append(x_c.astype(np.float32))
    med_tensors.append(x_d.astype(np.int8))

vital_tensors = np.stack(vital_tensors, axis=0)   # (N, 48, d_c)
med_tensors   = np.stack(med_tensors,   axis=0)   # (N, 48, d_d)
lengths       = np.array(lengths, dtype=np.int32)

print("✔️  continuous  :", vital_tensors.shape)
print("✔️  discrete    :", med_tensors.shape)
print("✔️  lengths     :", lengths.shape)


mean_c = vital_tensors.reshape(-1, vital_tensors.shape[-1]).mean(axis=0)
std_c  = vital_tensors.reshape(-1, vital_tensors.shape[-1]).std(axis=0)
std_c[std_c == 0] = 1.0  # safety

#  🔹 Normalize continuous tensors before saving
# vital_tensors = (vital_tensors - mean_c) / std_c

vital_file = f"vital_sign_{OUT_PREFIX}.pkl"
med_file   = f"med_interv_{OUT_PREFIX}.pkl"
stat_file  = "statics.pkl"          # empty (add metadata later if needed)
norm_file  = "norm_stats.npz"

with gzip.open(vital_file, "wb") as f:
    pickle.dump({"data": vital_tensors, "lengths": lengths}, f, protocol=4)

with gzip.open(med_file, "wb") as f:
    pickle.dump({"data": med_tensors, "lengths": lengths}, f, protocol=4)

with gzip.open(stat_file, "wb") as f:
    pickle.dump({}, f)              # placeholder

np.savez_compressed(norm_file,
                    mean=mean_c.astype("float32"),
                    std=std_c.astype("float32"),
                    feat_names=np.array(continuous_cols))

print("\n📦  Saved files and tensor shapes:")
print(f"   • {vital_file:25s} {os.path.getsize(vital_file)//1024:>6,} KB → shape: {vital_tensors.shape}")
print(f"   • {med_file:25s} {os.path.getsize(med_file)//1024:>6,} KB → shape: {med_tensors.shape}")
print(f"   • {stat_file:25s} {os.path.getsize(stat_file)//1024:>6,} KB → shape: empty dict")
print(f"   • {norm_file:25s} {os.path.getsize(norm_file)//1024:>6,} KB → mean/std: {mean_c.shape}")

print(f"\n➡️  Update your ehrMGAN config:")
print(f"   time_steps        = {T_MAX}")
print(f"   vital_sign_path   = '{vital_file}'")
print(f"   med_interv_path   = '{med_file}'")
print(f"   statics_path      = '{stat_file}'")
print(f"   norm_stat_path    = '{norm_file}'")
print(f"   dim               = {vital_tensors.shape[-1] + med_tensors.shape[-1]}")

📊 Merged hourly shape: (198338, 227) — rows: 198,338, columns: 227


packing:   0%|          | 0/471 [00:00<?, ?it/s]

✔️  continuous  : (471, 48, 29)
✔️  discrete    : (471, 48, 194)
✔️  lengths     : (471,)

📦  Saved files and tensor shapes:
   • vital_sign_48hrs.pkl         242 KB → shape: (471, 48, 29)
   • med_interv_48hrs.pkl          16 KB → shape: (471, 48, 194)
   • statics.pkl                    0 KB → shape: empty dict
   • norm_stats.npz                 1 KB → mean/std: (29,)

➡️  Update your ehrMGAN config:
   time_steps        = 48
   vital_sign_path   = 'vital_sign_48hrs.pkl'
   med_interv_path   = 'med_interv_48hrs.pkl'
   statics_path      = 'statics.pkl'
   norm_stat_path    = 'norm_stats.npz'
   dim               = 223


In [2]:
nulls = hourly.isna().sum()
if nulls.sum() > 0:
    print("🚨 NaNs still exist:")
    print(nulls[nulls > 0])
else:
    print("✅ No NaNs remain.")

✅ No NaNs remain.


In [1]:
%cd ehrMGAN

/hpc/home/ti45/tasnuba_sepsis/ehrMGAN


In [2]:
!python main_train.py --disease mydata --num_pre_epochs 500 --num_epochs 800 --epoch_ckpt_freq 100 --gpu 0

2025-07-24 14:04:04.288490: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1753380244.314248 3544174 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753380244.320416 3544174 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1753380244.335746 3544174 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1753380244.335785 3544174 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1753380244.335793 3544174 computation_placer.cc:177] computation placer alr

In [6]:
!pip install rarfile

Defaulting to user installation because normal site-packages is not writeable
